In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
from pickle import dump, load
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
import json
import requests

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Source

In [ ]:
tokenizer_filename = 'tokenizer.p'

In [ ]:
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''
  

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

In [ ]:
!sudo apt-get install tensorflow-model-server

In [ ]:
MODEL_DIR="crnn_model_19"
MODEL_DIR_tw="xception"

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=model_crnn \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8502 \
  --model_name=model_xception \
  --model_base_path="${MODEL_DIR_tw}" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
def extract_features_api(filename):
    try:
        image = Image.open(filename)
    except:
        print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
    image = image.resize((299,299))
    image = np.array(image)
    # for images that has 4 channels, we convert them into 3 channels
    if image.shape[2] == 4: 
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    # print(image.shape," Shape of input to server")
    data = json.dumps({"signature_name": "serving_default", "instances": image.tolist()})
    headers = {"content-type": "application/json"}
    json_response = requests.post('http://localhost:8504/v1/models/mdl:predict', data=data, headers=headers)
    predictions = json.loads(json_response.text)
    feature=np.array(predictions["predictions"])
    # print("Shape of response from server = ",feature.shape)
    
    return feature

In [ ]:
def generate_desc_api(tokenizer, photo, max_length):
    in_text = 'start'
    headers = {"content-type": "application/json"}
    # print(photo)
    # print("input for photo is ",photo.shape," type of photo is ",type(photo))
    for i in range(max_length):
        print(f"Requesting {i}th time ")
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        # print('\n',len([photo,sequence])," Shape of input to lstm\n")
        # print("Shape for sequence is ",sequence.shape," type of sequence  is ",type(sequence))
        in1=photo.tolist()
        in2=sequence.tolist()
        data = json.dumps({"signature_name":"serving_default","inputs": {'input_2':in1,'input_3':in2}})
        json_response = requests.post('http://localhost:8501/v1/models/mdlr:predict', data=data, headers=headers)
        predictions = json.loads(json_response.text)
        # print("Predicitons are ",predictions)
        res=predictions['outputs']
        pred_mdl=np.array(res)
        pred = np.argmax(pred_mdl)
        # print(pred,"THis is prediction")
        # pred = model.predict([photo,sequence], verbose=0)
        # print('Type of preds is ',type(pred))
        word = word_for_id(pred, tokenizer)
        # print("Outputted word is ",word)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [ ]:
def test_img_api(img_path):
  max_length = 32
  photo=extract_features_api(img_path)
  tokenizer = load(open(tokenizer_filename,"rb"))
  # xception_model = Xception(include_top=False, pooling="avg")
  img = Image.open(img_path)
  description = generate_desc_api(tokenizer, photo, max_length)
  print(description)
  plt.imshow(img)


In [ ]:
test_img_api('3637013_c675de7705.jpg')